# LangServeで遊ぶ

次のコマンドでWebアプリケーションを起動してからノートブックを実行していく。

```sh
poetry run task start
```

In [1]:
import requests
from datetime import datetime

base_url = "http://localhost:8000"

## とりあえずOpenAIでチャット

In [2]:
json = {
    "input": "日本の首都は？"
}
resp = requests.post(f"{base_url}/chat/invoke", json=json)
resp.json()

{'output': {'content': '東京です。',
  'additional_kwargs': {},
  'type': 'ai',
  'name': None,
  'id': None,
  'example': False},
 'callback_events': [],
 'metadata': {'run_id': '2f0754b0-2ea6-4c27-9c9a-0b4dfe78c9c3'}}

In [3]:
json = {
    "input": "日本の首都は？"
}
resp = requests.post(f"{base_url}/chat/stream", json=json, stream=True)
[line for line in resp.iter_lines(decode_unicode=True)]

['event: metadata',
 'data: {"run_id": "6b26bbb0-5b90-45aa-9b3e-67faeac5c603"}',
 '',
 'event: data',
 'data: {"content":"","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: data',
 'data: {"content":"東","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: data',
 'data: {"content":"京","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: data',
 'data: {"content":"です","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: data',
 'data: {"content":"。","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: data',
 'data: {"content":"","additional_kwargs":{},"type":"AIMessageChunk","name":null,"id":null,"example":false}',
 '',
 'event: end',
 '']

## `RunnablePassthrough`で入力と出力を確認してみる

In [4]:
json = {
    "input": "日本の首都は？"
}
resp = requests.post(f"{base_url}/passthrough/invoke", json=json)
resp.json()["output"]

'日本の首都は？'

In [5]:
json = {
    "input": {
        "question": "日本の首都は？"
    }
}
resp = requests.post(f"{base_url}/passthrough/invoke", json=json)
resp.json()["output"]

{'question': '日本の首都は？'}

In [6]:
json = {
    "input": [ "日本の首都は？" ]
}
resp = requests.post(f"{base_url}/passthrough/invoke", json=json)
resp.json()["output"]

['日本の首都は？']

In [7]:
resp = requests.post(f"{base_url}/passthrough/stream", json={ "input": "日本の首都は？" }, stream=True)
[line for line in resp.iter_lines(decode_unicode=True)]

['event: metadata',
 'data: {"run_id": "b3b3e5c1-0589-4066-8efe-dfa222ab26a4"}',
 '',
 'event: data',
 'data: "日本の首都は？"',
 '',
 'event: end',
 '']

## `RunnableLambda`

In [8]:
json = {
    "input": {
        "text": "abc",
        "times": 3,
    }
}
resp = requests.post(f"{base_url}/lambda/invoke", json=json)
resp.json()["output"]

'abcabcabc'

In [9]:
json = {
    "input": {
        "text": "abc",
        "times": 3,
    }
}
resp = requests.post(f"{base_url}/lambda/stream", json=json, stream=True)
[line for line in resp.iter_lines(decode_unicode=True)]

['event: metadata',
 'data: {"run_id": "8139d16d-d9c2-4a92-8322-e353afb27ca8"}',
 '',
 'event: data',
 'data: "abcabcabc"',
 '',
 'event: end',
 '']

## `RunnableGenerator`でServer-Sent Events

In [10]:
json = {
    "input": {
        "text": "hello",
        "sleep": 1,
    }
}
resp = requests.post(f"{base_url}/generator/stream", json=json, stream=True)
date = datetime.now()
for line in resp.iter_lines(decode_unicode=True):
    td = datetime.now() - date
    print((f"{td.seconds}", line))


('0', 'event: metadata')
('0', 'data: {"run_id": "792127e7-78f8-4c9a-9eb0-c1bbf0160d7a"}')
('0', '')
('0', 'event: data')
('0', 'data: "h"')
('0', '')
('1', 'event: data')
('1', 'data: "e"')
('1', '')
('2', 'event: data')
('2', 'data: "l"')
('2', '')
('3', 'event: data')
('3', 'data: "l"')
('3', '')
('4', 'event: data')
('4', 'data: "o"')
('4', '')
('5', 'event: end')
('5', '')


In [11]:
json = {
    "input": {
        "text": "hello",
        "sleep": 1,
    }
}
date = datetime.now()
resp = requests.post(f"{base_url}/generator/invoke", json=json, stream=True)
dt = datetime.now() - date
dt, resp.json()

(datetime.timedelta(seconds=5, microseconds=22577),
 {'output': 'hello',
  'callback_events': [],
  'metadata': {'run_id': 'e5750eaf-62a3-4f74-a1d4-b85d3dbf5578'}})

## チェーンしてみる

In [12]:
json = {
    "input": "hello",
}
resp = requests.post(f"{base_url}/chain1/stream", json=json, stream=True)
date = datetime.now()
for line in resp.iter_lines(decode_unicode=True):
    td = datetime.now() - date
    print((f"{td.seconds}", line))


('1', 'event: metadata')
('1', 'data: {"run_id": "f50ca9e9-3188-4b2e-853c-eaba9a8a2321"}')
('1', '')
('1', 'event: data')
('1', 'data: {"bar":"barhello1"}')
('1', '')
('2', 'event: data')
('2', 'data: {"foo":"foohello1"}')
('2', '')
('3', 'event: data')
('3', 'data: {"bar":"barhello2"}')
('3', '')
('3', 'event: data')
('3', 'data: {"foo":"foohello2"}')
('3', '')
('4', 'event: data')
('4', 'data: {"bar":"barhello3"}')
('4', '')
('4', 'event: data')
('4', 'data: {"foo":"foohello3"}')
('4', '')
('4', 'event: data')
('4', 'data: {"foobar":"foobar"}')
('4', '')
('4', 'event: end')
('4', '')


### 会話する

In [13]:
session_id = "e4d63ba5-627f-4854-b53e-5d7a83926319"
json = {
    "input": {
        "question": "日本の首都は？",
    },
    "config": {
        "configurable": {
            "session_id": session_id,
        },
    },
}
resp = requests.post(f"{base_url}/chatbot/invoke", json=json)
resp.json()

{'output': '東京です。',
 'callback_events': [],
 'metadata': {'run_id': 'b23ada58-3a1e-4faf-b0fe-54dcfe8f0d7a'}}

In [14]:
json = {
    "input": {
        "question": "イタリアは？",
    },
    "config": {
        "configurable": {
            "session_id": session_id,
        },
    },
}
resp = requests.post(f"{base_url}/chatbot/invoke", json=json)
resp.json()

{'output': 'ローマです。',
 'callback_events': [],
 'metadata': {'run_id': '6b8020f7-8d91-4a8b-86b2-0d924d9f44a1'}}

In [15]:
json = {
    "input": {
        "question": "フランスは？",
    },
    "config": {
        "configurable": {
            "session_id": session_id,
        },
    },
}
resp = requests.post(f"{base_url}/chatbot/invoke", json=json)
resp.json()

{'output': 'パリです。',
 'callback_events': [],
 'metadata': {'run_id': 'af601a92-95f5-4411-a12a-851dd4183a49'}}